In [1]:
import pandas as pd

# show all columns
pd.set_option('display.max_columns', None)

In [2]:
enigh_2012_mun=pd.read_csv('data/out/enigh_2012_mun_pol.csv')
enigh_2016_mun=pd.read_csv('data/out/enigh_2016_mun_pol.csv')

In [3]:
eceg_2010_mun = pd.read_csv('data/out/eceg_2010_mun.csv')

In [4]:
enigh_2012_mun.rename(columns={'CVE_MUN':'municipio','CVE_ENT':'entidad'}, inplace=True)   
enigh_2016_mun.rename(columns={'CVE_MUN':'municipio','CVE_ENT':'entidad'}, inplace=True)   

In [5]:
# join eceg and promedio_por_municipio
eceg_2010_gasto = pd.merge(enigh_2012_mun, eceg_2010_mun, on=['municipio', 'entidad'], how='inner')

In [6]:
#denue_2020_mun = pd.read_csv('data/out/denue_2020_mun.csv')

In [7]:
#eceg_2020_gasto_farm = pd.merge(eceg_2020_gasto, denue_2020_mun, on=['municipio', 'entidad'], how='inner')

In [8]:
#gasto en farmacias = medicinas + cuida_pers
enigh_2016_mun['gasto_farmacia'] = enigh_2016_mun['medicinas'] + enigh_2016_mun['cuida_pers']

In [9]:
XY = pd.merge(eceg_2010_gasto, enigh_2016_mun, on=["entidad","municipio"], how='inner')

In [11]:
XY = XY.drop(columns=['entidad','municipio','medicinas_y','cuida_pers_y'])

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# definir variables X y Y (predictores y variable a predecir)
X = XY.loc[:, ["medicinas_x","cuida_pers_x","pobtot", "p_0a2", "pder_ss","pocupada", "ing_cor"]]
Y = XY.loc[:, 'gasto_farmacia']

# estandarizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# separar entre datos de entrenamiento y datos de prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)


Y_train = Y_train.to_numpy().astype(np.float32)
Y_test = Y_test.to_numpy().astype(np.float32)


In [13]:
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import Dropout


# capas del modelo
model = Sequential()

model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(units=256, activation='relu', kernel_regularizer=l2(0.01)))  # L2 regularization
model.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(units=1, activation='linear'))

# cmpilar modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

In [14]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# reducir aprendizaje cuando no mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [15]:
# entrenamiento
history = model.fit(X_train, Y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 54000009216.0000 - mean_squared_error: 54000009216.0000 - val_loss: 12680058880.0000 - val_mean_squared_error: 12680058880.0000 - learning_rate: 0.0010
Epoch 2/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 42734080000.0000 - mean_squared_error: 42734080000.0000 - val_loss: 12670383104.0000 - val_mean_squared_error: 12670383104.0000 - learning_rate: 0.0010
Epoch 3/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 67154964480.0000 - mean_squared_error: 67154964480.0000 - val_loss: 12642902016.0000 - val_mean_squared_error: 12642902016.0000 - learning_rate: 0.0010
Epoch 4/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 42763038720.0000 - mean_squared_error: 42763038720.0000 - val_loss: 12581798912.0000 - val_mean_squared_error: 12581798912.0000 - learning_rate: 0.0010
Epoch 5/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 56520056832.0000 - mean_squared_error: 56520056832.0000 - val_loss: 12466452480.0000 - val_mean_s

In [16]:
from sklearn.metrics import r2_score

# hacer predicciones
Y_pred = model.predict(X_test)

# calcular R-squared
r2 = r2_score(Y_test, Y_pred)
print(f"R-squared: {r2}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
R-squared: 0.6157666444778442


In [17]:
X_test

array([[ 0.03131603, -0.77799353, -0.33247211, ..., -0.32335667,
        -0.30796078, -0.71472897],
       [-0.0828728 ,  0.33800907,  0.03878128, ...,  0.12351892,
        -0.06532441,  0.38402869],
       [-0.29688442,  0.79628045, -0.28327939, ..., -0.31338995,
        -0.26659045,  0.43911481],
       ...,
       [-0.11977646,  0.93996006, -0.30899959, ..., -0.30933452,
        -0.28933589,  0.69033623],
       [-0.22587619, -0.58264452, -0.05764307, ..., -0.15753109,
        -0.07268646, -0.43438512],
       [-0.70860484, -0.16131777,  0.40306533, ...,  0.32697745,
         0.3808855 , -0.42167785]])

In [18]:
print(Y_test[0:10])
print(Y_pred[0:10])

[ 28089.312  90434.02   20382.213  24875.09   28831.76   30532.596
  54318.99  142487.3    20664.26   15221.81 ]
[[ 37738.7  ]
 [ 88239.586]
 [ 83305.23 ]
 [ 40245.883]
 [ 33222.65 ]
 [ 34587.31 ]
 [135498.34 ]
 [269254.3  ]
 [107243.84 ]
 [ 34890.586]]


In [19]:
# guardar modelo
model.save('data/out/modelos/predictor_gasto_farmacia.keras')
